In [2]:
import os
import pandas as pd
import mysql.connector
from mysql.connector import Error
from dotenv import load_dotenv
from mistralai import Mistral

load_dotenv()

df = None
def get_connection():
    DB_USERNAME = os.getenv("DB_USERNAME")       
    DB_PASSWORD = os.getenv("DB_PASSWORD")   
    DB_HOST = os.getenv("DB_HOST")  
    DB_DATABASE = os.getenv("DB_DATABASE")     
    try:
        connection = mysql.connector.connect(
            host=DB_HOST,
            user=DB_USERNAME,
            password=DB_PASSWORD,
            database=DB_DATABASE
        )

        if connection.is_connected():
            return connection

    except Error as e:
        print(f"Error: {e}")
        return None

In [3]:
api_key = os.environ.get("MISTRAL_API_KEY")
model = "mistral-small-2506"

client = Mistral(api_key=api_key)

In [4]:
def generate_report(question, csv_string):
    report_prompt = f"""
    You are a report generation agent.

    Task:
    - Generate a crisp, data-driven report based only on the given context below.
    - Follow standard report steps: overview, key patterns, contributing factors, and conclusion.
    - Output ONLY pure Markdown content.
    - Do NOT wrap your response in code blocks (no ```markdown or ``` tags).
    - Do NOT include any preamble or explanation outside the report.
    - Start directly with the markdown heading.
    - Keep it concise and to the point.

    Context:
    Generate a report about: {question} based on the below data
    {csv_string}
    """

    chat_response = client.chat.complete(
        model=model,
        messages=[{"role": "user", "content": report_prompt}]
    )

    return chat_response.choices[0].message.content

In [6]:
connection = get_connection()
output_dir = "../public/ai/computed"
os.makedirs(output_dir, exist_ok=True)
metadata = [{"title":"What incidents involving fatalities or serious injuries occurred under ideal driving conditions (clear weather, daylight, dry roads, no work zone)",
             "query":"select * from report_1;",
             "filename":"report_1.md"},
             {"title":"Analyzing Contributing Factors Behind Work Zone Crashes",
              "query": "select * from report_2;",
              "filename":"report_2.md"},
             {"title":"Understanding Why Hit-and-Run Collisions Occur With Parked Motor Vehicles",
              "query":"select * from report_3",
              "filename":"report_3.md"},
             {"title":"Identifying High-Risk Locations With the Most Frequent Incident Occurrences",
              "query":"select * from report_4_2;",
              "filename":"report_4.md"},
            {"title":"Examining Traffic Control and Intersection Types in Severe or Fatal Crashes",
             "query":"select * from report_5;",
             "filename":"report_5.md"}
]
for _ in metadata[-2:]:
    query = _["query"]
    df = pd.read_sql(query, connection)
    csv_string = df.to_csv(index=False).strip()
    md = generate_report(_["title"], _["query"]) 
    out_path = os.path.join(output_dir, _["filename"])
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(md)


C:\Users\sathy\AppData\Local\Temp\ipykernel_12880\1298702086.py:22: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)
